# Recurrent Neural Networks: Application

In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
with open("dinos.txt", "r") as file:
    data = file.readlines()
    data = '<eos>'.join(data).lower()
    data = data.split('<eos>')
    random.shuffle(data)

chars = ''.join(data)
vocab = sorted(list(set(chars)))

In [3]:
data

['serendipaceratops\n',
 'peishansaurus\n',
 'pneumatoarthrus\n',
 'vulcanodon\n',
 'dimodosaurus\n',
 'coronosaurus\n',
 'shuangbaisaurus\n',
 'eugongbusaurus\n',
 'canardia\n',
 'megalosaurus\n',
 'kittysaurus\n',
 'saichania\n',
 'arcusaurus\n',
 'dandakosaurus\n',
 'siamodracon\n',
 'leptospondylus\n',
 'cheneosaurus\n',
 'opisthocoelicaudia\n',
 'gongpoquansaurus\n',
 'megadontosaurus\n',
 'adamantisaurus\n',
 'quilmesaurus\n',
 'notohypsilophodon\n',
 'diabloceratops\n',
 'pyroraptor\n',
 'loncosaurus\n',
 'aragosaurus\n',
 'mantellodon\n',
 'chuxiongosaurus\n',
 'brasileosaurus\n',
 'serikornis\n',
 'zhenyuanlong\n',
 'shidaisaurus\n',
 'mantellisaurus\n',
 'auroraceratops\n',
 'archaeornis\n',
 'yibinosaurus\n',
 'zupaysaurus\n',
 'vectensia\n',
 'adeopapposaurus\n',
 'velociraptor\n',
 'syntarsus\n',
 'pachyrhinosaurus\n',
 'byronosaurus\n',
 'avaceratops\n',
 'geminiraptor\n',
 'parrosaurus\n',
 'hierosaurus\n',
 'protorosaurus\n',
 'edmontonia\n',
 'trimucrodon\n',
 'xenopos

In [4]:
vocab

['\n',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z']

In [5]:
data_size = len(data)
vocab_size = len(vocab)

print('There are {} names and {} unique tokens in your data.'.format(data_size, vocab_size))

There are 1536 names and 27 unique tokens in your data.


In [6]:
char_to_idx = {ch:i for i,ch in enumerate(vocab)}
idx_to_char = {i:ch for i,ch in enumerate(vocab)}

In [7]:
char_to_idx

{'\n': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

In [8]:
idx_to_char

{0: '\n',
 1: 'a',
 2: 'b',
 3: 'c',
 4: 'd',
 5: 'e',
 6: 'f',
 7: 'g',
 8: 'h',
 9: 'i',
 10: 'j',
 11: 'k',
 12: 'l',
 13: 'm',
 14: 'n',
 15: 'o',
 16: 'p',
 17: 'q',
 18: 'r',
 19: 's',
 20: 't',
 21: 'u',
 22: 'v',
 23: 'w',
 24: 'x',
 25: 'y',
 26: 'z'}

In [9]:
max_length_name = max([len(d) for d in data])+5
eos_id = char_to_idx['\n']

X_train = np.zeros(shape=(data_size, max_length_name, vocab_size), dtype='float32')
for i, name in enumerate(data):
    for j, character in enumerate(name):
        index = char_to_idx[character]
        X_train[i,j,index] = 1.0
    X_train[i,len(name):,eos_id] = 1.0

Y_train = np.zeros(shape=(data_size, max_length_name, vocab_size), dtype='float32')
for i in range(data_size):
    for j in range(max_length_name-1):
        Y_train[i,j,:] = X_train[i,j+1,:]
    Y_train[i,max_length_name-1,eos_id] = 1.0

In [10]:
def Model(params):
    dims = params['dims']
    X_train = params['X_train']
    Y_train = params['Y_train']
    epochs = params['epochs']
    learning_rate = params['learning_rate']
    batch_size = params['batch_size']
    f1 = tf.keras.layers.SimpleRNN(units=50,
                                   activation='tanh',
                                   kernel_initializer='glorot_uniform',
                                   bias_initializer='zeros',
                                   recurrent_initializer='zeros',
                                   return_sequences=True, 
                                   name='rnn')
    f2 = tf.keras.layers.Dense(units=dims[1][1], 
                               activation='softmax', 
                               kernel_initializer='glorot_uniform', 
                               bias_initializer='zeros',
                               name='dense')
    x = tf.keras.Input(shape=dims[0])
    a = f1(x)
    y = f2(a)
    model = tf.keras.Model(x, y)    
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, 
                                         beta_1=0.9, 
                                         beta_2=0.999, 
                                         epsilon=1e-07)
    model.compile(loss='categorical_crossentropy', metrics=['categorical_accuracy'], optimizer=optimizer)
    model.summary()
    callback1 = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', 
                                                     patience=200,
                                                     min_delta=0.001,
                                                     factor=0.1, 
                                                     min_lr=0.0001)
    callback2 = tf.keras.callbacks.EarlyStopping(monitor='loss',
                                                 patience=500,
                                                 min_delta=0.001)
    model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size, callbacks=[callback1, callback2])
    return model

In [11]:
model = Model({'dims': [X_train.shape[1:], Y_train.shape[1:]], 
               'X_train': X_train,
               'Y_train': Y_train,
               'epochs':1000, 
               'learning_rate': 0.01,
               'batch_size': 64})

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 27)]          0         
_________________________________________________________________
rnn (SimpleRNN)              (None, 32, 50)            3900      
_________________________________________________________________
dense (Dense)                (None, 32, 27)            1377      
Total params: 5,277
Trainable params: 5,277
Non-trainable params: 0
_________________________________________________________________
Train on 1536 samples
Epoch 1/1000
1536/1536 [==============================] - 1s 542us/sample - loss: 1.3958 - categorical_accuracy: 0.6705
Epoch 2/1000
1536/1536 [==============================] - 0s 114us/sample - loss: 0.8216 - categorical_accuracy: 0.7672
Epoch 3/1000
1536/1536 [==============================] - 0s 117us/sample - loss: 0.7208 - categorical_accuracy: 0.7895
Epoch 4

1536/1536 [==============================] - 0s 143us/sample - loss: 0.5148 - categorical_accuracy: 0.8437
Epoch 63/1000
1536/1536 [==============================] - 0s 128us/sample - loss: 0.5115 - categorical_accuracy: 0.8442
Epoch 64/1000
1536/1536 [==============================] - 0s 124us/sample - loss: 0.5103 - categorical_accuracy: 0.8447
Epoch 65/1000
1536/1536 [==============================] - 0s 136us/sample - loss: 0.5085 - categorical_accuracy: 0.8453
Epoch 66/1000
1536/1536 [==============================] - 0s 133us/sample - loss: 0.5061 - categorical_accuracy: 0.8457
Epoch 67/1000
1536/1536 [==============================] - 0s 142us/sample - loss: 0.5045 - categorical_accuracy: 0.8454
Epoch 68/1000
1536/1536 [==============================] - 0s 149us/sample - loss: 0.5042 - categorical_accuracy: 0.8468
Epoch 69/1000
1536/1536 [==============================] - 0s 134us/sample - loss: 0.5027 - categorical_accuracy: 0.8465
Epoch 70/1000
1536/1536 [=====================

1536/1536 [==============================] - 0s 153us/sample - loss: 0.4573 - categorical_accuracy: 0.8586
Epoch 130/1000
1536/1536 [==============================] - 0s 141us/sample - loss: 0.4552 - categorical_accuracy: 0.8602
Epoch 131/1000
1536/1536 [==============================] - 0s 140us/sample - loss: 0.4547 - categorical_accuracy: 0.8597
Epoch 132/1000
1536/1536 [==============================] - 0s 135us/sample - loss: 0.4540 - categorical_accuracy: 0.8600
Epoch 133/1000
1536/1536 [==============================] - 0s 134us/sample - loss: 0.4528 - categorical_accuracy: 0.8604
Epoch 134/1000
1536/1536 [==============================] - 0s 132us/sample - loss: 0.4543 - categorical_accuracy: 0.8586
Epoch 135/1000
1536/1536 [==============================] - 0s 130us/sample - loss: 0.4515 - categorical_accuracy: 0.8604
Epoch 136/1000
1536/1536 [==============================] - 0s 139us/sample - loss: 0.4540 - categorical_accuracy: 0.8602
Epoch 137/1000
1536/1536 [=============

1536/1536 [==============================] - 0s 137us/sample - loss: 0.4386 - categorical_accuracy: 0.8637
Epoch 197/1000
1536/1536 [==============================] - 0s 129us/sample - loss: 0.4376 - categorical_accuracy: 0.8656
Epoch 198/1000
1536/1536 [==============================] - 0s 134us/sample - loss: 0.4397 - categorical_accuracy: 0.8635
Epoch 199/1000
1536/1536 [==============================] - 0s 136us/sample - loss: 0.4390 - categorical_accuracy: 0.8636
Epoch 200/1000
1536/1536 [==============================] - 0s 134us/sample - loss: 0.4411 - categorical_accuracy: 0.8640
Epoch 201/1000
1536/1536 [==============================] - 0s 131us/sample - loss: 0.4436 - categorical_accuracy: 0.8623
Epoch 202/1000
1536/1536 [==============================] - 0s 134us/sample - loss: 0.4371 - categorical_accuracy: 0.8650
Epoch 203/1000
1536/1536 [==============================] - 0s 134us/sample - loss: 0.4386 - categorical_accuracy: 0.8644
Epoch 204/1000
1536/1536 [=============

1536/1536 [==============================] - 0s 136us/sample - loss: 0.4343 - categorical_accuracy: 0.8649
Epoch 264/1000
1536/1536 [==============================] - 0s 128us/sample - loss: 0.4301 - categorical_accuracy: 0.8664
Epoch 265/1000
1536/1536 [==============================] - 0s 131us/sample - loss: 0.4293 - categorical_accuracy: 0.8671
Epoch 266/1000
1536/1536 [==============================] - 0s 129us/sample - loss: 0.4389 - categorical_accuracy: 0.8647
Epoch 267/1000
1536/1536 [==============================] - 0s 134us/sample - loss: 0.4338 - categorical_accuracy: 0.8646
Epoch 268/1000
1536/1536 [==============================] - 0s 130us/sample - loss: 0.4303 - categorical_accuracy: 0.8667
Epoch 269/1000
1536/1536 [==============================] - 0s 130us/sample - loss: 0.4295 - categorical_accuracy: 0.8663
Epoch 270/1000
1536/1536 [==============================] - 0s 134us/sample - loss: 0.4325 - categorical_accuracy: 0.8652
Epoch 271/1000
1536/1536 [=============

...

1536/1536 [==============================] - 0s 124us/sample - loss: 0.3772 - categorical_accuracy: 0.8836
Epoch 733/1000
1536/1536 [==============================] - 0s 125us/sample - loss: 0.3771 - categorical_accuracy: 0.8836
Epoch 734/1000
1536/1536 [==============================] - 0s 129us/sample - loss: 0.3769 - categorical_accuracy: 0.8839
Epoch 735/1000
1536/1536 [==============================] - 0s 121us/sample - loss: 0.3772 - categorical_accuracy: 0.8835
Epoch 736/1000
1536/1536 [==============================] - 0s 124us/sample - loss: 0.3768 - categorical_accuracy: 0.8835
Epoch 737/1000
1536/1536 [==============================] - 0s 122us/sample - loss: 0.3770 - categorical_accuracy: 0.8837
Epoch 738/1000
1536/1536 [==============================] - 0s 125us/sample - loss: 0.3807 - categorical_accuracy: 0.8822
Epoch 739/1000
1536/1536 [==============================] - 0s 120us/sample - loss: 0.3827 - categorical_accuracy: 0.8819
Epoch 740/1000
1536/1536 [=============

1536/1536 [==============================] - 0s 122us/sample - loss: 0.3736 - categorical_accuracy: 0.8848
Epoch 800/1000
1536/1536 [==============================] - 0s 121us/sample - loss: 0.3735 - categorical_accuracy: 0.8849
Epoch 801/1000
1536/1536 [==============================] - 0s 120us/sample - loss: 0.3733 - categorical_accuracy: 0.8852
Epoch 802/1000
1536/1536 [==============================] - 0s 120us/sample - loss: 0.3730 - categorical_accuracy: 0.8845
Epoch 803/1000
1536/1536 [==============================] - 0s 120us/sample - loss: 0.3735 - categorical_accuracy: 0.8850
Epoch 804/1000
1536/1536 [==============================] - 0s 129us/sample - loss: 0.3729 - categorical_accuracy: 0.8853
Epoch 805/1000
1536/1536 [==============================] - 0s 122us/sample - loss: 0.3733 - categorical_accuracy: 0.8848
Epoch 806/1000
1536/1536 [==============================] - 0s 124us/sample - loss: 0.3742 - categorical_accuracy: 0.8843
Epoch 807/1000
1536/1536 [=============

1536/1536 [==============================] - 0s 121us/sample - loss: 0.3700 - categorical_accuracy: 0.8857
Epoch 867/1000
1536/1536 [==============================] - 0s 120us/sample - loss: 0.3702 - categorical_accuracy: 0.8863
Epoch 868/1000
1536/1536 [==============================] - 0s 121us/sample - loss: 0.3705 - categorical_accuracy: 0.8858
Epoch 869/1000
1536/1536 [==============================] - 0s 124us/sample - loss: 0.3708 - categorical_accuracy: 0.8856
Epoch 870/1000
1536/1536 [==============================] - 0s 126us/sample - loss: 0.3708 - categorical_accuracy: 0.8854
Epoch 871/1000
1536/1536 [==============================] - 0s 123us/sample - loss: 0.3709 - categorical_accuracy: 0.8854
Epoch 872/1000
1536/1536 [==============================] - 0s 120us/sample - loss: 0.3707 - categorical_accuracy: 0.8857
Epoch 873/1000
1536/1536 [==============================] - 0s 121us/sample - loss: 0.3708 - categorical_accuracy: 0.8859
Epoch 874/1000
1536/1536 [=============

1536/1536 [==============================] - 0s 121us/sample - loss: 0.3692 - categorical_accuracy: 0.8856
Epoch 934/1000
1536/1536 [==============================] - 0s 122us/sample - loss: 0.3695 - categorical_accuracy: 0.8863
Epoch 935/1000
1536/1536 [==============================] - 0s 122us/sample - loss: 0.3697 - categorical_accuracy: 0.8859
Epoch 936/1000
1536/1536 [==============================] - 0s 122us/sample - loss: 0.3692 - categorical_accuracy: 0.8859
Epoch 937/1000
1536/1536 [==============================] - 0s 122us/sample - loss: 0.3698 - categorical_accuracy: 0.8857
Epoch 938/1000
1536/1536 [==============================] - 0s 120us/sample - loss: 0.3696 - categorical_accuracy: 0.8860
Epoch 939/1000
1536/1536 [==============================] - 0s 127us/sample - loss: 0.3695 - categorical_accuracy: 0.8865
Epoch 940/1000
1536/1536 [==============================] - 0s 119us/sample - loss: 0.3695 - categorical_accuracy: 0.8860
Epoch 941/1000
1536/1536 [=============

1536/1536 [==============================] - 0s 132us/sample - loss: 0.3668 - categorical_accuracy: 0.8871


In [12]:
for i in range(100):
    name = ''
    for idx in np.argmax(X_train[i,:,:], axis=1):
        name = name + idx_to_char[idx]
        if idx==0:
            break
    name_prediction = '' + name[0]
    for idx in np.argmax(model.predict(X_train)[i,:,:], axis=1):
        name_prediction = name_prediction + idx_to_char[idx]
        if idx==0:
            break
    print('Original Name: {} - Predicted Name: {}'.format(name.split()[0], name_prediction.split()[0]))

Original Name: serendipaceratops - Predicted Name: sacanoittleratops
Original Name: peishansaurus - Predicted Name: palbtansaurus
Original Name: pneumatoarthrus - Predicted Name: paeumatorlaoais
Original Name: vulcanodon - Predicted Name: veaormoson
Original Name: dimodosaurus - Predicted Name: dalaconaurus
Original Name: coronosaurus - Predicted Name: cheysasaurus
Original Name: shuangbaisaurus - Predicted Name: saaengaegsaurus
Original Name: eugongbusaurus - Predicted Name: eurnngjusaurus
Original Name: canardia - Predicted Name: chmlsoia
Original Name: megalosaurus - Predicted Name: magamosaurus
Original Name: kittysaurus - Predicted Name: kotaosaurus
Original Name: saichania - Predicted Name: sausoinoa
Original Name: arcusaurus - Predicted Name: anchlaurus
Original Name: dandakosaurus - Predicted Name: dastacosaurus
Original Name: siamodracon - Predicted Name: sanmodracon
Original Name: leptospondylus - Predicted Name: lapeosaondylus
Original Name: cheneosaurus - Predicted Name: ch

-----

In [13]:
def InferenceModel(params):
    dims = params['dims']
    model = params['model']
    f1 = tf.keras.layers.SimpleRNN(units=50,
                                  activation='tanh',
                                  return_sequences=True,
                                  return_state=True, 
                                  name='rnn')
    f2 = tf.keras.layers.Lambda(lambda x: x[0])
    f3 = tf.keras.layers.Lambda(lambda x: x[1])
    f4 = tf.keras.layers.Dense(units=dims[1], 
                               activation='softmax', 
                               name='dense')
    x0 = tf.keras.Input(shape=(1,dims[0]))
    a0 = tf.keras.Input(shape=(50))
    z0 = f1([x0,a0])
    x1 = f2(z0)
    a1 = f3(z0)
    x2 = f4(x1)
    inference_model = tf.keras.Model([x0,a0], [x2,a1]) 
    inference_model.get_layer(name='rnn').set_weights(model.get_layer(name='rnn').get_weights())
    inference_model.get_layer(name='dense').set_weights(model.get_layer(name='dense').get_weights())
    return inference_model

In [14]:
inference_model = InferenceModel({'dims': [X_train.shape[2], Y_train.shape[2]],
                                  'model': model})

In [15]:
num_names = 30
num_samples = max_length_name + 10
print('Sampling names...')
for i in range(num_names):
    x = np.zeros(shape=(1,1,vocab_size))
    a = np.zeros(shape=(1,50))
    word = ''
    for _ in range(num_samples):
        z = inference_model.predict([x, a])
        idx = np.random.choice(range(vocab_size), p = z[0].ravel())
        word = word + idx_to_char[idx]
        if idx==0:
            break
        x = np.zeros(shape=(1,1,vocab_size), dtype='float32')
        x[0,0,idx] = 1.0
        a = z[1]
    print('Generated name {}/{}: {}'.format(i+1, num_names, word.split()[0]))

Sampling names...
Generated name 1/30: ebechnosaurus
Generated name 2/30: ankerorna
Generated name 3/30: uenisaurus
Generated name 4/30: engesaurus
Generated name 5/30: uewtogosaurus
Generated name 6/30: anawasaurus
Generated name 7/30: abienchasaurus
Generated name 8/30: aeasceritosaurus
Generated name 9/30: icernodon
Generated name 10/30: amnanosaurus
Generated name 11/30: ancoelisti
Generated name 12/30: abbenssaursausaurus
Generated name 13/30: inosaurus
Generated name 14/30: iniangosaurus
Generated name 15/30: raptorniptetis
Generated name 16/30: abzerachus
Generated name 17/30: uerypensa
Generated name 18/30: oloraterys
Generated name 19/30: errosia
Generated name 20/30: aeilodon
Generated name 21/30: ilaniang
Generated name 22/30: canosaurus
Generated name 23/30: ankerodon
Generated name 24/30: aloskus
Generated name 25/30: ichiosaurus
Generated name 26/30: uectopcopieus
Generated name 27/30: eratosaurus
Generated name 28/30: uenidonafanosaurus
Generated name 29/30: uengheda
Gen